In [189]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [190]:
df = pd.read_csv('pred_INE_data.csv')
df1 = pd.read_csv('INE_data.csv')

df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.set_index('Date', inplace=True)

df1['Date'] = pd.to_datetime(df1['Date'], format='%Y-%m-%d')
df1.set_index('Date', inplace=True)


df_final = pd.concat([df,df1],axis=0)

In [191]:
def predict_value(df, columna, city):
    df_temp = df.copy()
    
    X_num = pd.DataFrame(df_temp[columna])
    standarizer = StandardScaler().fit(X_num)
    x_standarized = standarizer.transform(X_num)
    df_temp[columna] = x_standarized
    
    df_city = df_temp[df_temp['city'] == city].sort_index()

    train_data = df_city[columna]['2017-01-01':'2024-12-01']
    train_data = train_data[~train_data.index.duplicated(keep='first')]
    
    train_data.shape
    # Configurar y entrenar el modelo SARIMA
    sarima_model = SARIMAX(train_data,
                        order=(0, 1, 1),  # p, d, q
                        seasonal_order=(0, 1, 1, 12),  # P, D, Q, s (12 para datos mensuales)
                        enforce_stationarity=False,
                        enforce_invertibility=False)

    # Ajustar el modelo
    sarima_results = sarima_model.fit(disp=False)
    forecast = sarima_results.get_forecast(steps=12)
    forecast_values = forecast.predicted_mean
    values = forecast_values.reset_index()
    values['predicted_mean'] = values['predicted_mean'].apply(lambda x : round(standarizer.inverse_transform([[x]])[0][0], 2))
    return values

def predict_merge(df, city):
    pern = predict_value(df, 'pernoctaciones',city)
    viajeros = predict_value(df, 'viajeros',city)
    precios = predict_value(df, 'precios_hoteleros',city)
    ipc = predict_value(df,'ipc_restaurantes_hoteles',city)

    df_merged = pd.merge(pern, viajeros, on='index')
    df_merged.rename(columns={'predicted_mean_x' : 'pernoctaciones', 'predicted_mean_y' : 'viajeros'}, inplace=True)

    df_merged = pd.merge(df_merged, precios, on='index')
    df_merged = pd.merge(df_merged, ipc, on='index')
    df_merged.rename(columns={'predicted_mean_x' : 'precios_hoteleros', 'predicted_mean_y' : 'ipc_restaurantes_hoteles'}, inplace=True)

    df_merged['city'] = city
    df_merged['index'] = pd.to_datetime(df_merged['index']).dt.strftime('%Y-%m-%d')

    df_merged.set_index('index', inplace=True)
    
    return df_merged


In [192]:
predict_merge(df_final,0)

c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\sta

,pernoctaciones,viajeros,precios_hoteleros,ipc_restaurantes_hoteles,city
index,,,,,
2025-01-01,3735910.67,661310.20,145.94,120.31,0
2025-02-01,3763426.89,728682.85,143.45,120.99,0
2025-03-01,3795460.89,742418.28,146.03,121.69,0
2025-04-01,4022826.60,917856.71,146.20,122.90,0
2025-05-01,4455228.88,992692.85,190.88,123.35,0
2025-06-01,5806904.42,1237391.13,174.22,124.20,0
2025-07-01,8914548.52,1651886.26,172.81,124.72,0
2025-08-01,10599289.22,1799724.97,175.01,125.06,0
2025-09-01,6558928.98,1300689.70,165.17,125.07,0


In [193]:
df_final.index

DatetimeIndex(['2023-11-01', '2023-10-01', '2023-09-01', '2023-08-01',
               '2023-07-01', '2023-06-01', '2023-05-01', '2023-04-01',
               '2023-03-01', '2023-02-01',
               ...
               '2017-10-01', '2017-09-01', '2017-08-01', '2017-07-01',
               '2017-06-01', '2017-05-01', '2017-04-01', '2017-03-01',
               '2017-02-01', '2017-01-01'],
              dtype='datetime64[ns]', name='Date', length=3222, freq=None)

In [194]:
df_prediction = df_final.copy()
df_prediction = df_prediction[0:0]
df_prediction

,city,pernoctaciones,viajeros,precios_hoteleros,ipc_restaurantes_hoteles
Date,,,,,


In [ ]:
for i in range(0,18,1):
    df_prediction = pd.concat([df_prediction, predict_merge(df_final,i)], axis=0)


In [202]:
df_prediction.reset_index(names='Date', inplace=True)
df_prediction.set_index('Date', inplace=True)

df_final.index = pd.to_datetime(df_final.index)
df_prediction.index = pd.to_datetime(df_prediction.index)

df_data = pd.concat([df_final, df_prediction], axis=0)

In [211]:
df_data.sort_index(inplace=True)
df_data.reset_index(inplace=True)
df_data.to_csv('predict.csv', index=False)